In [1]:
%load_ext autoreload
%autoreload 2
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs, DecodingArgs

target_dir = proj_root() / "../SPOT-copy/"
model_path = get_model_dir(True) / "coeditor-large-query-stub"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
batch_args = BatchArgs.service_default()
service = EditPredictionService(
    target_dir,
    model,
    batch_args=batch_args,
    encoder=BasicQueryEditEncoder(
        max_ref_tks=batch_args.max_ref_tks,
        max_query_tks=batch_args.max_query_tks,
        max_output_tks=batch_args.max_output_tks,
    ),
    dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
)


In [4]:
service.suggest_edit(Path("src/coeditor/retrieval_model.py"), 1062)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -0.09068
* Modified: 
    ... 2 lines omitted ...
        @dataclass
        class _BatchSampler:
            def __iter__(self) -> Iterable[Mapping]:
                if self.shuffle:
                    for es in self.edit_groups:
                        random.shuffle(es)
    -           batches = edits_to_batches(self.edit_groups, self.data_args)
    +           batches = edit_groups_to_batches(self.edit_groups, self.data_args)
                if self.shuffle:
                    random.shuffle(batches)
 
                tqdm_args = self.tqdm_args or {"smoothing": 0.0}
                for b in tqdm(batches, desc=self.desc + f" {self.epochs}", **tqdm_args):
                    input_ids = pad_token_seqs(b["input_ids"])
    ... 8 lines omitted ...
==

In [5]:
service.suggest_edit(Path("src/coeditor/common.py"), 292)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -0.4088
* Modified: 
        # edit: coeditor.common/random_subset
    -   def random_subset(all, n: int, seed: int = 42):
    +   def random_subset(all, n: int, rng: random.Random | None = None):
            if isinstance(all, Sequence):
                xs = [x for x in all]
    -           random.Random(seed).shuffle(xs)
    +           rng.shuffle(xs)
                return xs[:n]
            elif isinstance(all, Mapping):
                keys = [k for k in all]
    -           random.Random(seed).shuffle(keys)
    +           rng.shuffle(keys)
                return {k: all[k] for k in keys[:n]}
            else:
                raise ArgumentError(all, f"Unsupported arg type: {type(all)}")
========== Sugeestion 1 ==========
score: -2.718
* Modifi

In [6]:
service.suggest_edit(Path("src/coeditor/encoders.py"), 292)

stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.encoders', 'coeditor.retrieval_model'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Sugeestion 0 ==========
score: -2.145
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def has_change(tks: TokenSeq) -> bool:
    -       return Add_id in tks or Del_id in tks
    +   def has_change(tks: TokenSeq, skip_prefix: int = 16) -> bool:
    +       return Add_id in tks or Del_id in tks[skip_prefix:]
========== Sugeestion 1 ==========
score: -3.025
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def has_change(tks: TokenSeq) -> bool:
    -       return Add_id in tks or Del_id in tks
    +   def has_change(tks: TokenSeq, skip_prefix: int = 16) -> bool:
    +       return Add_id in tks or Del_id in tks or skip_prefix in tks
========== Sugeestion 2 ==========
score: -3.137
* Modified: 
        # edit: coeditor.encoders/has_change
    -   def 

In [ ]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
1,construct project edit,1,3.296461,3.296461
3,run model,1,1.625003,1.625003
2,encode edits,1,0.676539,0.676539
0,get target element,1,0.158115,0.158115
